## **Embedding Generation Using UNI (Unified Image Embedding Model)**

In this notebook, we will:
- Load the pre-processed and augmented images.
- Generate embeddings using the UNI model.
- Save the embeddings and related data for evaluation..

---

## **Table of Contents**
---
1. Import Libraries
2. Set Up Device
3. Load Augmented Image Mapping
4. Load UNI Model
5. Generate Embeddings
6. Select Representative Embeddings
7. Save Embeddings
8. Clear Memory
9. Conclusion

---
### **Step 1: Import Libraries**

We begin by importing the necessary libraries.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import transforms
import gc

# Add UNI directory to sys.path
sys.path.append('./UNI')

# Import the necessary modules from UNI
from uni.get_encoder import get_encoder

---
### **Step 2: Set Up Device**

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


---
### **Step 3: Load Augmented Image Mapping**

In [3]:
# Load augmented image mapping
augmented_df = pd.read_csv('augmented_image_mapping.csv')

---
### **Step 4: Load UNI Model**

In [5]:
# Step 4: Load UNI Model

# Load the pre-trained UNI model
checkpoint = 'pytorch_model.bin'

# Use the get_encoder function to load the model and the transform
model, transform = get_encoder(
    enc_name='vit_large_patch16_224.dinov2.uni_mass100k',
    checkpoint=checkpoint,
    which_img_norm='imagenet',
    img_resize=224,
    center_crop=False,
    test_batch=0,
    device=device,
    assets_dir=assets_dir,
    kwargs={},
)

---
### **Step 5: Generate Embeddings**

In [7]:
batch_size = 168  # Adjust based on your GPU memory
embeddings = []
original_image_files = []

# Directory containing augmented images
augmented_dir = './augmented_dataset/'

# Prepare a list of all image paths and corresponding original images
image_paths = augmented_df['augmented_image'].apply(lambda x: os.path.join(augmented_dir, x)).tolist()
original_images = augmented_df['original_image'].tolist()

# Total number of images
total_images = len(image_paths)

# Generate embeddings in batches
for start_idx in tqdm(range(0, total_images, batch_size), desc='Generating Embeddings'):
    end_idx = min(start_idx + batch_size, total_images)
    batch_image_paths = image_paths[start_idx:end_idx]
    batch_original_images = original_images[start_idx:end_idx]

    # Load and preprocess images
    imgs = [Image.open(p).convert('RGB') for p in batch_image_paths]
    img_tensors = torch.stack([transform(img) for img in imgs]).to(device)

    # Generate embeddings
    with torch.no_grad():
        batch_embeddings = model(img_tensors)
        batch_embeddings = batch_embeddings.cpu().numpy()
        # Normalize embeddings
        batch_embeddings = batch_embeddings / np.linalg.norm(batch_embeddings, axis=1, keepdims=True)
        embeddings.extend(batch_embeddings)
        original_image_files.extend(batch_original_images)

    # Clear memory
    del imgs, img_tensors, batch_embeddings
    torch.cuda.empty_cache()
    gc.collect()

Generating Embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [4:58:52<00:00, 99.63s/it]


---
### **Step 6: Select Representative Embeddings**

We will use the Highest Norm Criterion to select the most representative embedding for each original image.

In [8]:
# Convert embeddings to NumPy array
embeddings = np.vstack(embeddings)

# Create a DataFrame for grouping
data = pd.DataFrame({
    'original_image': augmented_df['original_image'],
    'embedding_index': range(len(embeddings))
})

selected_embeddings = []
selected_image_files = []

# Group by original image and select the embedding with the highest norm
for original_image, group in data.groupby('original_image'):
    indices = group['embedding_index'].tolist()
    group_embeddings = embeddings[indices]
    # Compute norms of embeddings
    norms = np.linalg.norm(group_embeddings, axis=1)
    # Select the embedding with the highest norm
    best_idx_in_group = np.argmax(norms)
    best_idx = indices[best_idx_in_group]
    selected_embeddings.append(embeddings[best_idx])
    selected_image_files.append(original_image)

---
### **Step 7: Save Embeddings**

In [9]:
# Convert selected embeddings to NumPy array
selected_embeddings = np.vstack(selected_embeddings)

# Save embeddings and image files
np.save('embeddings_uni.npy', selected_embeddings)
np.save('image_files_uni.npy', selected_image_files)

print('Embeddings for UNI saved.')

Embeddings for UNI saved.


---
### **Step 8: Clear Memory**

In [10]:
# Clear variables and free memory
del embeddings, augmented_image_files, original_image_files, data
del selected_embeddings, selected_image_files, model
torch.cuda.empty_cache()
gc.collect()

0

---
### **Conclusion**

We have generated embeddings using the UNI model, selected representative embeddings, and saved them for evaluation.